In [ ]:
import os
os.chdir('/Users/jjaniak/Documents/studia/projekt/gradient')

import geopandas as gpd
import pandas as pd
import json

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import shap
import numpy as np

from shapely.wkt import loads

from esda.moran import Moran_Local
import libpysal

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from src.embedders.osm_data_embedder import OSMDataEmbedder
from srai.regionalizers import geocode_to_region_gdf
from srai.embedders import CountEmbedder
from srai.regionalizers import H3Regionalizer
from srai.loaders.osm_loaders.filters import OsmTagsFilter

from IPython.display import display

pd.set_option("display.max_columns", None)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


json_folder_path = "notebooks/accident_analysis/out/data_analysis_results/"

In [ ]:
city_name = "Pozna\u0144"
nominatim_city_name = "Pozna\u0144, Poland"
year = 2022

## Functions

In [ ]:
query: OsmTagsFilter = {"highway": True, "railway": True, "route": True, "amenity": True}

def rename_columns_for_saving(hex_and_features_gdf):
    # Define a mapping for renaming columns
    column_mapping = {
        "highway": "H_",
        "railway": "R_",
        "amenity": "A_",
        "route": "RO_",
    }

    # Rename columns based on the defined mapping
    for prefix, replacement in column_mapping.items():
        columns_to_rename = [col for col in hex_and_features_gdf.columns if col.startswith(prefix + "_")]
        new_column_names = [replacement + col[len(prefix) + 1:] for col in columns_to_rename]
        hex_and_features_gdf.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)

    return hex_and_features_gdf

def create_hex_gds(h3_resolution, city_name=city_name):
    data_embedder = OSMDataEmbedder(
        area=geocode_to_region_gdf(nominatim_city_name),
        embedder=CountEmbedder(),
        regionalizer=H3Regionalizer(resolution=h3_resolution),
        query=query,
    )
    filename = f"data/baseline-datasets/in/{city_name}-hex-res-{h3_resolution}-and-features_with_amenity-gdf_fixed_names.shp"
    if not os.path.exists(filename):
        hex_and_features_gdf = data_embedder.make_embeddings()  # type: ignore
        hex_and_features_gdf_fixed_names = rename_columns_for_saving(hex_and_features_gdf)
        hex_and_features_gdf_fixed_names.to_file(
            filename,
            index=True,
        )
    else:
        hex_and_features_gdf = gpd.read_file(filename)
        hex_and_features_gdf.set_index("region_id", inplace=True)

    return hex_and_features_gdf


def get_accidents_gdf(h3_resolution, city_name=city_name, year=year):
    filename = f"data/accidents_in_hex/{city_name}_accidents_{year}_res{h3_resolution}.csv"
    if not os.path.exists(filename):
        raise FileNotFoundError(f"The file {filename} does not exist.")
    else:
        accidents_df = pd.read_csv(filename)
        accidents_df['geometry'] = accidents_df['geometry'].apply(loads)
        accidents_gdf = gpd.GeoDataFrame(accidents_df, geometry='geometry', crs="EPSG:4326")
        return accidents_gdf
    
def merge_gdf(resolution):
    filename = f"data/accidents_in_hex/{city_name}_accidents_and_features_{year}_res{resolution}.shp"
    if not os.path.exists(filename):
        hex_and_features_gdf = create_hex_gds(h3_resolution=resolution, city_name=city_name)
        accidents_gdf = get_accidents_gdf(h3_resolution=resolution, city_name=city_name, year=year)
        merged_gdf = gpd.sjoin(left_df=accidents_gdf, right_df=hex_and_features_gdf, how='inner', op='intersects')
        merged_gdf = merged_gdf.drop(columns='index_right')
        merged_gdf['num_accidents'] = merged_gdf['count'].astype(int)
        merged_gdf.drop(columns=['count'], inplace=True)
        merged_gdf['binary_accidents'] = 0  # Initialize with 0
        merged_gdf.loc[merged_gdf["num_accidents"] > 0, "binary_accidents"] = 1.0
        merged_gdf.to_file(
            filename,
            index=True,
        )
    else:
        merged_gdf = gpd.read_file(filename)
        merged_gdf.set_index("region_id", inplace=True)
    
    merged_gdf.rename(columns={'num_accide': 'num_accidents', 'binary_acc': 'binary_accidents'}, inplace=True)
    return merged_gdf

In [ ]:
def local_moran(df, column):
    w = libpysal.weights.Queen.from_dataframe(df)
    y = df[column].values
    moran_loc = Moran_Local(y, w)
    return moran_loc.Is, moran_loc.p_sim, moran_loc.q

In [ ]:
from enum import Enum


class FeatureGoups(Enum):
    HIGHWAY = "H"
    RAILWAY = "R"
    AMENITY = "A"
    ROUTE  = "RO"
    

def analyze_features(merged_gdf:pd.DataFrame, feature_group: FeatureGoups):
    
    # Filter columns starting with feature group name
    feature_group_columns = merged_gdf.columns[merged_gdf.columns.str.startswith(feature_group.value)]

    # Sum values
    sum_feature_values = merged_gdf[feature_group_columns].sum()
    
    if sum_feature_values.size == 1:
        print(f"Found only one feature for feature group {feature_group.name}")
        display(sum_feature_values)
        return []
        
    else:
        plt.figure(figsize=(10, 6))
        sns.histplot(sum_feature_values, bins=50, kde=True, color='skyblue', edgecolor='black')
        plt.title(f'Histogram of {feature_group.name} Features')
        plt.xlabel('Sum')
        plt.ylabel('Frequency')
        plt.show()

        quartiles = sum_feature_values.quantile([0.25, 0.5, 0.75])

        iqr = quartiles[0.75] - quartiles[0.25]
        lower_bound = quartiles[0.25] - 1.5 * iqr
        upper_bound = quartiles[0.75] + 1.5 * iqr

        most_common_features = list(sum_feature_values[(sum_feature_values > upper_bound)].index)
        least_common_features = list(sum_feature_values[(sum_feature_values < lower_bound)].index)
        display(sum_feature_values.describe())
        print(f"\nMost common features for {feature_group.name}: \n {most_common_features}")
        if least_common_features:
            print(f"\nLeast common features for {feature_group.name}: \n {least_common_features}")
        
        return most_common_features
    
    
def analyze_imbalance(merged_gdf, feature_group):
    feature_group_columns = list(merged_gdf.columns[merged_gdf.columns.str.startswith(feature_group.value)]) + ['binary_accidents']
    grouped_data = merged_gdf[feature_group_columns].groupby('binary_accidents').sum()
    percentage_data = grouped_data.div(grouped_data.sum(axis=0))
        
    imbalance_ratio = abs(grouped_data.diff(axis=0) / grouped_data.sum(axis=0)).round(2)
    imbalance_ratio = imbalance_ratio.loc[1]
    
    print(f"\nMetrics of Imbalance in {feature_group.name}")
    if imbalance_ratio.size > 1:
        display(imbalance_ratio.T.describe())
    else:
        display(imbalance_ratio)
    
    high_imbalance_features = imbalance_ratio[imbalance_ratio >= 0.5]
    if high_imbalance_features.size > 0:
        print(f"\nThere are {len(high_imbalance_features)} high imbalance features in {feature_group.name}")
        for feature in high_imbalance_features.index:
            print(feature)
        display(percentage_data[high_imbalance_features.index])

        return high_imbalance_features.index
    else:
        return []

In [ ]:
def correlation_analysis(merged_gdf, resolution):
    
    all_features = merged_gdf.drop(columns=['geometry', 'region_id', 'num_accidents', 'binary_accidents']).columns.to_list()
    
    correlation_matrix = merged_gdf[all_features + ['binary_accidents']].corr()
    correlation_values = correlation_matrix['binary_accidents']

    correlation_metrics = correlation_values.drop('binary_accidents').describe()

    # Filter features based on IQR 
    high_corr_features = correlation_values[(correlation_values >= 0.2) | (correlation_values <= -0.2)].index.to_list()
    low_corr_features = correlation_values[(correlation_values < 0.2) & (correlation_values > -0.2)].index.to_list()

    high_corr_features.remove('binary_accidents')
    correlation_matrix_best_features = merged_gdf[high_corr_features + ['binary_accidents']].corr()

    if high_corr_features != []:
        plt.figure(figsize=(14, 12))
        sns.heatmap(correlation_matrix_best_features, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={'size': 6})
        plt.title(f'Correlation Heatmap (best features) for resolution = {resolution}')
        plt.show()

    return high_corr_features, low_corr_features, correlation_metrics

In [ ]:
def shap_analysis(h3_resolution, merged_gdf, selected_features, city_name=city_name, save_force_plot=False):
    print(f"Analysis for Resolution = {h3_resolution}")
    
    y = merged_gdf['binary_accidents']
    X = merged_gdf[selected_features]

    feature_names = X.columns.tolist()
    
    X_train, X_test, y_train, _ = train_test_split(X, y, test_size=0.3, random_state=42)
    
    scaler = preprocessing.StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=feature_names)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=feature_names)
    
    linear_lr = LogisticRegression(max_iter=1000)
    linear_lr.fit(X_train_scaled, y_train)

    #Przygotowanie przybliżonej próbki tła za pomocą metody k-means.
    background_summary = shap.kmeans(X_train, 10)

    explainer = shap.KernelExplainer(linear_lr.predict_proba, background_summary)

    #Obliczenie SHAP wartości dla danych testowych (X_test), co pozwala na zrozumienie, jak każda zmienna przyczynia się do przewidywanej wartości.
    shap_values = explainer.shap_values(X_test_scaled)

    shap.summary_plot(shap_values, X_test_scaled)

    if save_force_plot:
        shap.initjs()
        p = shap.force_plot(explainer.expected_value[0], shap_values[0], X_test_scaled)
        shap.save_html(json_folder_path + f'{city_name}_res_{h3_resolution}_force_plot_best_features.html', p)

## Analysis

In [ ]:
# Set resolutions
resolutions = [8, 9, 10]

# Initialize dataframe and dictionaries to store results
distribution_results = pd.DataFrame(columns=['Resolution', 'Num_Hexes', 'Accidents_Mean', 'Accidents_Std','Binary_Accidents_Mean', 'Binary_Accidents_Std'])

local_moran_results = {} 
highest_imbalance_features_dict = {}


# Loop through resolutions
for resolution in resolutions:
    
    print(f"Analysis for Resolution = {resolution}")
    
    # Create hex dataframes
    merged_gdf = merge_gdf(resolution)
    
    # Accidents Distribution
    new_row = {
        'Resolution': resolution,
        'Num_Hexes': len(merged_gdf),
        'Accidents_Mean': merged_gdf['num_accidents'].mean(),
        'Accidents_Std': merged_gdf['num_accidents'].std(),
        'Binary_Accidents_Mean': merged_gdf['binary_accidents'].mean(),
        'Binary_Accidents_Std': merged_gdf['binary_accidents'].std()
    }
    result_df = pd.DataFrame([new_row], columns=distribution_results.columns)
    distribution_results = pd.concat([distribution_results, result_df], ignore_index=True)

    # Calculate Local Moran's I
    moran_i, p_sim, q = local_moran(merged_gdf, column='num_accidents')
    local_moran_results[resolution] = {'moran_i': moran_i, 'p_sim': p_sim, 'q': q}

### Rozkład wypadków

In [ ]:
print("Distribution of Accidents in Hexes:")
display(distribution_results) 

#### Wnioski
- Na niższych rozdzielczościach heksagony mają tendencję do grupowania większej liczby wypadków.
- Odchylenie standardowe liczby wypadków maleje wraz ze wzrostem rozdzielczości. 
- Odchylenie standardowe na skali binarnej jest największe dla rozdzielczości 8 (0.49), co sugeruje większe zróżnicowanie między heksagonami pod względem obecności lub braku wypadków.


### Correlation

In [ ]:
df_correlation_metrics = pd.DataFrame()

all_high_corr_features = {}
all_low_corr_features = {}

# Loop through resolutions
for resolution in resolutions:
    print(f"Analysis for Resolution = {resolution}")
    merged_gdf = merge_gdf(resolution)
    merged_gdf.reset_index(inplace=True)
    highest_corr_features, low_corr_features, correlation_metrics = correlation_analysis(merged_gdf, resolution)
    all_low_corr_features[resolution] = low_corr_features
    all_high_corr_features[resolution] = highest_corr_features
    df_correlation_metrics = pd.concat([df_correlation_metrics, correlation_metrics], axis=1)


file_path = json_folder_path + f'all_high_corr_features_{city_name}.json'
with open(file_path, 'w') as json_file:
    json.dump(all_high_corr_features, json_file)
    
file_path = json_folder_path + f'all_low_corr_features_{city_name}.json'
with open(file_path, 'w') as json_file:
    json.dump(all_low_corr_features, json_file)

In [ ]:
df_correlation_metrics.columns = [f'resolution={resolution}' for resolution in resolutions]
print("Correlation Metrics of Binary Accidents with All Features:")
display(df_correlation_metrics)

In [ ]:
file_path = json_folder_path + f'all_high_corr_features_{city_name}.json'
with open(file_path, 'r') as json_file:
    all_high_corr_features = json.load(json_file)
    
print("Best Features Based on Correlation:\n")
for resolution, best_features in all_high_corr_features.items():
    print(f"\nResolution {resolution}:")
    for feature in best_features:
        print(f"{feature}")

In [ ]:
common_features = set(all_high_corr_features['8']) & set(all_high_corr_features['9']) & set(all_high_corr_features['10'])
if common_features != []:
    print("Common Features With High Correlation Across All Resolutions:")
    for feature in common_features:
        print(feature)
    
common_features = set(all_high_corr_features['8']) & set(all_high_corr_features['9'])
if common_features != []:
    print("Common Features With High Correlation Across Resolutions 8 and 9:")
    for feature in common_features:
        print(feature)

In [ ]:
file_path = json_folder_path + f'all_low_corr_features_{city_name}.json'
with open(file_path, 'r') as json_file:
    all_low_corr_features = json.load(json_file)


common_features = set(all_low_corr_features['8']) & set(all_low_corr_features['9']) & set(all_low_corr_features['10'])
if common_features != []:
    print("Common Features With Low Correlation Across All Resolutions:")
    for feature in common_features:
        print(feature)

### Istotne cechy

In [ ]:
file_path = json_folder_path + f'all_high_corr_features_{city_name}.json'
with open(file_path, 'r') as json_file:
    all_high_corr_features = json.load(json_file)
    
# Loop through resolutions
for resolution in resolutions:
    print(f"Analysis for Resolution = {resolution}")
    merged_gdf = merge_gdf(resolution)
    merged_gdf.reset_index(inplace=True)
    highest_corr_features = all_high_corr_features[str(resolution)]
    
    # Summary statistics of numerical columns
    summary_statistics = merged_gdf.describe()

    all_high_imbalance_features = []
    
    feature_groups = [FeatureGoups.AMENITY, FeatureGoups.HIGHWAY, FeatureGoups.RAILWAY, FeatureGoups.ROUTE]
    for feature_group in feature_groups:
        feature_group_columns = list(merged_gdf.columns[merged_gdf.columns.str.startswith(feature_group.value)])
        mean_row = summary_statistics[feature_group_columns].loc['mean']
        sorted_columns = mean_row.sort_values().index
        summary_statistics_sorted = summary_statistics[sorted_columns]
        print(f"\nSummary Statistics of Numerical Columns (Sorted by Mean) for {feature_group.name}:")
        display(summary_statistics_sorted)
        all_high_imbalance_features.extend(analyze_imbalance(merged_gdf, feature_group))
        highest_imbalance_features_dict[resolution] = all_high_imbalance_features
        

    filtered_columns = summary_statistics.columns[summary_statistics.loc['mean'] > 1]
    common_features_mean_more_than_1 = set(filtered_columns) & set(highest_corr_features)
    if common_features_mean_more_than_1:
        print(f"\nCommon features with mean in hex more than 1 and high correlation:")
        print(common_features_mean_more_than_1)
    else:
        print(f"\nThere is no common features with mean in hex more than 1 and high correlation.")
        
    filtered_columns = summary_statistics.columns[summary_statistics.loc['mean'] < 0.01]
    common_features_mean_less_than_001 = set(filtered_columns) & set(highest_corr_features)
    if common_features_mean_less_than_001:
        print(f"\nCommon features with mean in hex less than 0.01 and high correlation:")
        print(common_features_mean_less_than_001)
    else:
        print(f"\nThere is no common features with mean in hex less than 0.01 and high correlation.")
        
    common_features_highest_corr_and_imbalance = set(all_high_imbalance_features) & set(highest_corr_features)
    if common_features_highest_corr_and_imbalance:
        print(f"\nCommon features with high imbalance and high correlation:")
        print(common_features_highest_corr_and_imbalance)
    else:
        print(f"\nThere is no common features with high imbalance and high correlation.")
    

file_path = json_folder_path + f'highest_imbalance_features_dict_{city_name}.json'
with open(file_path, 'w') as json_file:
    json.dump(highest_imbalance_features_dict, json_file)

In [ ]:
features_resolution_8 = highest_imbalance_features_dict[8]
features_resolution_9 = highest_imbalance_features_dict[9]
features_resolution_10 = highest_imbalance_features_dict[10]

# Find common features
common_features = set(features_resolution_8) & set(features_resolution_9) & set(features_resolution_10)
if common_features != []:
    print("Common Features With the Highest Imbalance Across All Resolutions:")
    for feature in common_features:
        print(feature)

### Najczęściej występujące cechy

In [ ]:
# resolution=8
# merged_gdf = merge_gdf(resolution)

all_most_common_features = []
feature_groups = [FeatureGoups.AMENITY, FeatureGoups.HIGHWAY, FeatureGoups.RAILWAY, FeatureGoups.ROUTE]
for feature_group in feature_groups:
    all_most_common_features.extend(analyze_features(merged_gdf, feature_group))
    
print(f"\nAll Most Common Features in the City:")
for feature in all_most_common_features:
    print(feature)

### SHAP

In [ ]:
json_file_path = f'data/data_analysis_results/all_high_corr_features_{city_name}.json'    
with open(json_file_path, 'r') as json_file:
    all_high_corr_features = json.load(json_file)
    
for resolution, highest_corr_features in all_high_corr_features.items():
    if highest_corr_features != []:
        merged_gdf = merge_gdf(resolution)
        shap_analysis(h3_resolution=resolution, merged_gdf=merged_gdf, selected_features=highest_corr_features, save_force_plot=True)
    

### Moran's I

In [ ]:
print("Local Moran's I Results:")
for resolution, results in local_moran_results.items():
    print(f"Resolution {resolution}: Moran's I = {results['moran_i']}, p_sim = {results['p_sim']}, q = {results['q']}")

In [ ]:
res = {}
for resolution, results in local_moran_results.items():
    res[resolution] = []
    for key, value in results.items():
        res[resolution].extend([np.mean(value), np.std(value), np.min(value), np.max(value)])

local_moran_metrics = list(local_moran_results.values())[0].keys()
id_names = [f'{key}_{stat}' for key in local_moran_metrics for stat in ['mean', 'std', 'min', 'max']]
        
df = pd.DataFrame(res, index=id_names)
df.columns = [f'resolution={resolution}' for resolution in df.columns]
display(df)

#### Wnioski

**moran_i** <br>
 Dla rozdzielczości 8 (największe hexy) mamy silniejszy lokalny wzorzec korelacji niż dla rozdzielczości 9 i 10. Wyższe wartości wskazują na obszary, gdzie wartości są podobne do swoich sąsiadów, tworząc lokalne skupiska. <br>
 Odchylenie jest największe dla rozdzielczości 10, więc istnieje duża zmienność korelacji przestrzennej między heksagonami.
 Dla rozdzielczości 10, gdzie mamy największe wartości maksymalne, istnieją obszary o bardzo silnej lokalnej korelacji przestrzennej, co może oznaczać klastry podobnych wartości.

**p-wartość** <br>
 Im niższa wartość, tym silniejszy jest dowód na lokalną przestrzenną korelację. Dla wszystkich rozdzielczości średnie wartości p-sim są stosunkowo niskie, co wskazuje na istotność statystyczną lokalnego wzorca przestrzennej korelacji. <br>
 Dla największych wartości p_sim (bliskich 0.5) może oznaczać, że lokalny wskaźnik Morana nie jest istotny statystycznie w tych obszarach. Wartości te mogą pochodzić z obszarów o losowym rozkładzie, gdzie brak jest wyraźnych lokalnych wzorców. Niemniej jednak, wartości minimalne i wartość średnia sugerują, że dla większości obszarów lokalne wzorce korelacji są istotne.

**Q** <br>
 Wartości Q określają, do której kategorii kwadrantu należy dany obszar w analizie lokalnego Moran'a I. Średnie wartości dla wszystkich rozdzielczości wskazują na to, że większość obszarów należy do kwadrantów 1, 2 lub 3, co oznacza, że istnieją klastry obszarów o podobnych wartościach w sąsiedztwie.  <br>
 Kwadrant 4 oznacza obszary o wysokich wartościach otoczone przez obszary o niskich wartościach. Dla rozdzielczości 8 i 9 maksymalna wartość Q wynosi 4, co oznacza, że niektóre obszary są otoczone przez obszary o podobnych wartościach. Dla rozdzielczości 10 maksymalna wartość Q wynosi 3, co znaczy, że obszary o wyższych wartościach są bardziej izolowane lub mają bardziej zróżnicowane sąsiedztwo.